In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import warnings

In [2]:
dt = pd.read_csv('cub.csv', parse_dates=['date'])

In [3]:
bcda = {
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}
df = dt.resample('15T',on = 'date').agg(bcda)
df = df.between_time('09:15', '15:25')
df.reset_index(inplace=True)
df.head()

,date,open,high,low,close,volume
0,2021-06-15 09:15:00,171.00,171.55,170.30,170.85,151284
1,2021-06-15 09:30:00,170.70,170.95,169.60,170.00,111066
2,2021-06-15 09:45:00,170.00,170.35,169.85,169.95,59090
3,2021-06-15 10:00:00,169.95,170.00,169.80,169.95,38067
4,2021-06-15 10:15:00,169.95,170.00,169.60,169.65,30630


In [4]:
df.shape

(1604, 6)

In [5]:
atr = pd.DataFrame()
atr['hl'] = df['high']-df['low']
atr['hc'] = abs(df["high"]-df["close"].shift())
atr['lc'] = abs(df["low"]-df["close"].shift())
atr['atr'] = atr[['hl','hc','lc']].max(axis=1)
df["ATR"] = atr['atr'].rolling(window=14).mean()
df.head(20)

,date,open,high,low,close,volume,ATR
0,2021-06-15 09:15:00,171.00,171.55,170.30,170.85,151284,NaN
1,2021-06-15 09:30:00,170.70,170.95,169.60,170.00,111066,NaN
2,2021-06-15 09:45:00,170.00,170.35,169.85,169.95,59090,NaN
3,2021-06-15 10:00:00,169.95,170.00,169.80,169.95,38067,NaN
4,2021-06-15 10:15:00,169.95,170.00,169.60,169.65,30630,NaN
5,2021-06-15 10:30:00,169.70,169.85,169.50,169.55,23707,NaN
6,2021-06-15 10:45:00,169.55,169.70,169.00,169.10,106909,NaN
7,2021-06-15 11:00:00,169.10,169.90,169.05,169.75,32200,NaN
8,2021-06-15 11:15:00,169.75,169.95,169.65,169.75,29588,NaN
9,2021-06-15 11:30:00,169.65,169.95,169.60,169.75,6175,NaN


In [6]:
breech_time1 = []
bid_time1 = []
bid_price1 =[]
bid_trigger1=[]
Target1=[]
Stoploss1=[]
Sl_trigger_buy1=[]
entry_price1=[]
entry_time1=[]
exit_price1=[]
exit_time1=[]
exit_cause =[]
fractall_time=[]
fractall_price=[]
new_atr = []
will = pd.DataFrame()
fractal_up = [None,None]

In [7]:
def qwe(entry_time,entry_price):
    breech_time1.append(breech_time)
    bid_time1.append(bid_time)
    bid_price1.append(bid_price)
    bid_trigger1.append(bid_trigger)
    Target1.append(Target)
    Stoploss1.append(Stoploss)
    Sl_trigger_buy1.append(Sl_trigger_buy)
    entry_price1.append(entry_price)
    entry_time1.append(entry_time)
    fractall_price.append(fractal_price)
    fractall_time.append(fractal_time)
    new_atr.append(n_atr)
def new_exit(j,index,a,entry_time,entry_price):
    for n in range(a+1,len(dt)):
        if dt['date'][n]<datetime.datetime.combine(dt['date'][n].date(),datetime.time(15,15)):
            if dt['high'][n] > Target:
                exit_time1.append(dt['date'][n])
                exit_price = dt['high'][n]
                exit_price1.append(exit_price)
                qwe(entry_time,entry_price)
                index = j
                break
            if dt['low'][n] < Sl_trigger_buy:
                qwe(entry_time,entry_price)
                exit_time1.append(dt['date'][n])
                exit_price = dt['high'][n]
                exit_price1.append(exit_price)
                index = j
                break
        else:
                qwe(entry_time,entry_price)
                exit_time1.append(dt['date'][n-1])
                exit_price = dt['high'][n-1]
                exit_price1.append(exit_price)
                index = j
                break
    return index

def new_entry(j,index):
    for k in range(len(dt)):
        if dt['date'][k] == bid_time:
            for m in range(k,k+31):
                a5 =  dt['date'][m]>= datetime.datetime.combine(dt['date'][m].date(),datetime.time(9,30))
                a6 =  dt['date'][m]< datetime.datetime.combine(dt['date'][m].date(),datetime.time(14,30))
                a12 = dt['high'][m] >bid_trigger
                if a12 and a5 and a6:
                    entry_time = dt['date'][m]
                    entry_price = bid_price
                    index=new_exit(j,index,m,entry_time,entry_price)
                    break
                else:
                    entry_time=None
                    entry_price=None
    return index
count =0
i=2
index=2
while(i<len(df)-2):
        z=index
        a1=df['high'][i-2]<df['high'][i]
        a2=df['high'][i-1]<df['high'][i]
        a3=df['high'][i+2]<df['high'][i]
        a4=df['high'][i+1]<df['high'][i]
        if a1 and a2  and a3  and a4 :
            fractal_price = df['high'][i]
            fractal_time = df['date'][i]
            for p in range(i+1,len(df)-2):
                a7=df['high'][p-2]<df['high'][p]
                a8=df['high'][p-1]<df['high'][p]
                a9=df['high'][p+2]<df['high'][p]
                a10=df['high'][p+1]<df['high'][p]
                if a7  and a8 and a9 and a10 :
                    for j in range(i,p+1):
                        if df['high'][j] > fractal_price :
                            breech_time = df['date'][j]
                            bid_time = df['date'][j] + datetime.timedelta(0,900)
                            n_atr=df['ATR'][j]
                            bid_price = df['high'][j] + .15*df['ATR'][j]
                            bid_trigger = df['high'][j] + .10*df['ATR'][j]
                            Target = df['high'][j] + 2*df['ATR'][j]
                            Stoploss = df['high'][j]- 1*df['ATR'][j]
                            Sl_trigger_buy = Stoploss + .05*df['ATR'][j]
                            count+=1
                            index=new_entry(j,index)
                            break
                    break
        i+=1
        if z!=index:
            z = index
            i = index+1

In [8]:
will['fractal_time'] = fractall_time
will['fractal_price']= fractall_price
will['breech_time'] = breech_time1 
will['bid_time']=bid_time1
will['bid_price']=bid_price1
will['bid_trigger']=bid_trigger1
will['Target']=Target1
will['Stoploss']=Stoploss1
will['Sl_trigger_buy']=Sl_trigger_buy1
will['entry_price']=entry_price1
will['entry_time']=entry_time1
will['exit_price']=exit_price1
will['exit_time']=exit_time1
will['N_atr'] = new_atr

In [9]:
will

,fractal_time,fractal_price,breech_time,bid_time,bid_price,bid_trigger,Target,Stoploss,Sl_trigger_buy,entry_price,entry_time,exit_price,exit_time,N_atr
0,2021-06-16 12:15:00,167.95,2021-06-16 13:45:00,2021-06-16 14:00:00,168.446429,168.414286,169.635714,167.707143,167.739286,168.446429,2021-06-16 14:02:00,168.15,2021-06-16 15:04:00,0.642857
1,2021-06-23 09:15:00,165.15,2021-06-23 10:00:00,2021-06-23 10:15:00,165.654464,165.619643,166.942857,164.853571,164.888393,165.654464,2021-06-23 10:24:00,165.45,2021-06-23 10:27:00,0.696429
2,2021-06-23 11:45:00,166.50,2021-06-23 13:30:00,2021-06-23 13:45:00,166.878214,166.852143,167.842857,166.278571,166.304643,166.878214,2021-06-23 14:07:00,166.55,2021-06-23 14:18:00,0.521429
3,2021-06-29 10:15:00,170.80,2021-06-29 11:30:00,2021-06-29 11:45:00,171.160714,171.107143,173.142857,169.928571,169.982143,171.160714,2021-06-29 11:45:00,170.20,2021-06-29 14:40:00,1.071429
4,2021-07-05 14:30:00,165.40,2021-07-06 09:15:00,2021-07-06 09:30:00,165.888393,165.858929,166.978571,165.210714,165.240179,165.888393,2021-07-06 09:43:00,167.05,2021-07-06 09:45:00,0.589286
5,2021-07-06 10:45:00,166.85,2021-07-06 13:45:00,2021-07-06 14:00:00,168.282500,168.255000,169.300000,167.650000,167.677500,168.282500,2021-07-06 14:01:00,168.15,2021-07-06 14:25:00,0.550000
6,2021-07-07 13:30:00,165.45,2021-07-08 09:15:00,2021-07-08 09:30:00,165.964821,165.943214,166.764286,165.467857,165.489464,165.964821,2021-07-08 09:31:00,165.80,2021-07-08 09:40:00,0.432143
7,2021-07-15 09:30:00,163.30,2021-07-15 10:30:00,2021-07-15 10:45:00,163.420714,163.397143,164.292857,162.878571,162.902143,163.420714,2021-07-15 10:45:00,164.45,2021-07-15 11:17:00,0.471429
8,2021-07-29 14:30:00,150.95,2021-07-30 09:15:00,2021-07-30 09:30:00,151.301786,151.267857,152.557143,150.521429,150.555357,151.301786,2021-07-30 09:43:00,150.80,2021-07-30 14:51:00,0.678571
9,2021-08-03 14:00:00,152.25,2021-08-04 09:15:00,2021-08-04 09:30:00,154.438929,154.409286,155.535714,153.757143,153.786786,154.438929,2021-08-04 09:30:00,155.55,2021-08-04 09:46:00,0.592857
